<h1 align='center'> Chatbot - Tamil

In [ ]:
# !pip install torch==1.3.1+cpu -f https://download.pytorch.org/whl/torch_stable.html
# !pip install inltk

In [ ]:
import io
import random
import string 
import warnings
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
from prompt_toolkit import print_formatted_text,HTML

###############  nltk  ####################
import nltk
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
nltk.download('popular', quiet=True) # for downloading packages

###############  inltk  ####################
from inltk.inltk import predict_next_words
from inltk.inltk import get_similar_sentences
from inltk.inltk import get_sentence_similarity
import re

from inltk.inltk import setup
from inltk.inltk import tokenize
from inltk.inltk import get_embedding_vectors


warnings.filterwarnings('ignore')

In [ ]:
# setup('ta')

### Examples inltk

In [ ]:
example_sent = "உங்களைப் பார்த்து நிறைய நாட்கள் ஆகிவிட்டது"# Predict next 'n' tokens
n = 5
pred_sent = predict_next_words(example_sent, n, 'ta')# Get 'n' similar sentence
n = 2
simi_sent = get_similar_sentences(example_sent, n, 'ta')

In [ ]:
print("Predicted Words:", pred_sent)
print("Similar Sentences:", simi_sent)

### Read Text file.

In [ ]:
f=open('MG-Hector_Details.txt','r',errors = 'ignore',encoding='utf8')
raw=f.read()

### Tokenization:

In [ ]:
sent_tokens = nltk.sent_tokenize(raw)# converts to list of sentences 
word_tokens = nltk.word_tokenize(raw)# converts to list of words

### Greetings

In [ ]:
GREETING_INPUTS = ("வணக்கம்", "வாழ்த்துக்கள்", "ஏய்", "ஹாய்")
GREETING_RESPONSES = ["ஹாய்", "வணக்கம்","வாருங்கள்", "ஹாய், நீங்கள் எப்படி இருக்கிறீர்கள்"]
def greeting(sentence):
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

### Function for getting response for the user questions.

In [ ]:
def GetResponse(user_response):
    robo_response=''
    #Append the use response to the sentence tokens
    sent_tokens.append(user_response)
    TfidfVec = TfidfVectorizer()
    #find the tfidf vectors for the sentence tokens
    tfidf = TfidfVec.fit_transform(sent_tokens)
    #find the cosine similarity btw the appended word and all other words in the sentence tokens
    vals = cosine_similarity(tfidf[-1], tfidf)
    #sort the score of similarities by leaving the last appended sentence score.
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    # Check for highest cosine similarity score.
    if(req_tfidf==0):
        robo_response=robo_response+"என்னை மன்னித்துவிடு! நீங்கள் என்ன கேட்கிறீர்கள் என்று எனக்கு புரியவில்லைை"
        return robo_response
    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response

### Chat

In [ ]:
flag=True
print_formatted_text(HTML("<b>உதவியாளர் : </b>"),"ஹாய், எம்ஜி ஹெக்டர் கார் பற்றிய விவரங்களை என்னால் கொடுக்க முடியும். நீங்கள் வெளியேற விரும்பினால், வருகிறேன் என்று தட்டச்சு செய்க!")
print("\n")
while(flag==True):
    user_response = input("நீங்கள் : ")
    if(user_response!='வருகிறேன்'):
        if(user_response=='நன்றி' or user_response=='thank you' ):
            flag=False
            print_formatted_text(HTML("<b>உதவியாளர் : </b>"),"நீங்கள் வரவேற்கப்படுகிறீர்கள்..")
            print("\n")
        else:
            if(greeting(user_response)!=None):
                print_formatted_text(HTML("<b>உதவியாளர் : </b>"),greeting(user_response))
                print("\n")
            else:
                print_formatted_text(HTML("<b>உதவியாளர் : </b>"),end="")
                print(GetResponse(user_response))
                print("\n")
                sent_tokens.remove(user_response)
    else:
        flag=False
        print_formatted_text(HTML("<b>உதவியாளர் : </b>"),"நன்றி, மீண்டும் வாருங்கள்")